<a href="https://colab.research.google.com/github/chaitraDev/GenAI-for-marketing/blob/main/Cognizant_Recommendation%26GenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
customers = pd.read_excel("/content/drive/MyDrive/Cognizant/cust_clustered_nonum.xlsx")

In [2]:
def get_customers(product_name):
  # Find the clusters that have this product
  clusters = customers[customers.Product == product_name].Clusters_ID.unique()
  prod_index = customers[customers.Product == product_name].index

  # Find how many times the product has appeared in those clusters
  cluster_product = customers.loc[:,["Clusters_ID","Product"]]
  mapping = {cluster:0 for cluster in clusters}
  for index in prod_index:
    cluster_no = cluster_product.iloc[index]["Clusters_ID"]
    mapping[cluster_no] += 1
  print(mapping)

  # find the cluster that has this product max number of times
  best_cluster = sorted(mapping.items(),key=lambda pair:pair[1],reverse=True)[0][0]

  # Find the customers that belong to this cluster and aren't using this product
  targets = customers[((customers.Clusters_ID == best_cluster) & (customers.Product!=product_name))]
  return targets

In [3]:
targets = get_customers("National_Pension_System")
targets.head()

{40: 980, 21: 977, 32: 821, 5: 560, 28: 455, 46: 1125}


,CustomerID,RFM_Score,Relationship_Value,Age,Cibil_Score,Income,Product,Clusters_ID
33,C6931388,512,high,94,502,200.0,Senior Citizens Health Insurance,46
35,C5331380,412,high,94,445,300.0,Senior Citizens Health Insurance,46
55,C2514473,412,high,93,517,134.0,Senior Citizens Health Insurance,46
60,C7625952,514,high,93,601,731.0,Senior Citizens Health Insurance,46
63,C4014424,515,high,93,673,1973.0,Senior Citizens Health Insurance,46


# GenAI

In [4]:
import google.generativeai as genai
from google.colab import userdata
from IPython.display import Markdown,display

In [5]:
api = userdata.get('api')

In [6]:
genai.configure(api_key = api)
model = genai.GenerativeModel('gemini-1.5-flash',system_instruction="You are sales team lead in bank")

In [7]:
hot_lead_id = targets[targets.Relationship_Value == "high"]["CustomerID"].iloc[0]

In [8]:
customer = customers[customers.CustomerID == hot_lead_id]
recommendation = "National_Pension_System"
prompt = f"""
            Given this customer profile (Age: {customer.Age} yrs, Cibil Score: {customer.Cibil_Score},
            RFM score: {customer.RFM_Score}, Previous Product: {customer.Product})
            State why {recommendation} would be good recommendation for this customer.
            Answer in tabular format with following columns:
            'Parameter' - (Strength Features's Value),'Strengths', 'Benefit of {recommendation}', 'Recommendations', and 'Important Considerations'
            Just include bullet points with only limited keywords; no explanations.
          """

In [9]:
response = model.generate_content(prompt)

In [10]:
display(Markdown((response.text)))

## Recommendation for National Pension System (NPS)

| Parameter | Strengths | Benefit of National_Pension_System | Recommendations | Important Considerations |
|---|---|---|---|---|
| **Age: 33 years** | - Young and has time to build wealth | - Long-term wealth accumulation | -  Start contributing early to maximize returns | -  NPS is a long-term investment; may not be suitable for immediate financial needs |
| **Cibil Score: 502** | - Good credit score | -  Access to higher NPS contribution limits |  - Explore higher contribution options | -  Maintaining good credit score is crucial for NPS eligibility |
| **RFM Score: 512** | - High value customer | -  Potential for higher returns through consistent contributions |  -  Offer attractive schemes with potential for higher returns |  -  Ensure understanding of investment risks |
| **Previous Product: Senior Citizens Health Insurance** |  - Existing customer |  -  Opportunity for cross-selling |  -  Highlight NPS benefits for long-term financial security |  -  Consider customer's existing financial portfolio | 
